# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)


In [4]:
df = df.dropna(thresh=10).drop(['original', 'child_alone'], axis=1)

In [5]:
X = df.iloc[:,1]
y = df.iloc[:,5:]

In [6]:
for i in y.columns:
    print (i)
    print (y[i].unique())

offer
[ 0.  1.]
aid_related
[ 0.  1.]
medical_help
[ 0.  1.]
medical_products
[ 0.  1.]
search_and_rescue
[ 0.  1.]
security
[ 0.  1.]
military
[ 0.  1.]
water
[ 0.  1.]
food
[ 0.  1.]
shelter
[ 0.  1.]
clothing
[ 0.  1.]
money
[ 0.  1.]
missing_people
[ 0.  1.]
refugees
[ 0.  1.]
death
[ 0.  1.]
other_aid
[ 0.  1.]
infrastructure_related
[ 0.  1.]
transport
[ 0.  1.]
buildings
[ 0.  1.]
electricity
[ 0.  1.]
tools
[ 0.  1.]
hospitals
[ 0.  1.]
shops
[ 0.  1.]
aid_centers
[ 0.  1.]
other_infrastructure
[ 0.  1.]
weather_related
[ 0.  1.]
floods
[ 0.  1.]
storm
[ 0.  1.]
fire
[ 0.  1.]
earthquake
[ 0.  1.]
cold
[ 0.  1.]
other_weather
[ 0.  1.]
direct_report
[ 0.  1.]


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
def model_pipeline(x):

    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(x, n_jobs=-1))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = model_pipeline(KNeighborsClassifier(n_neighbors=10, n_jobs=-1))
model.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._params=None, n_jobs=-1, n_neighbors=10, p=2,
           weights='uniform'),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = model.predict(X_test)
print (classification_report(y_test, y_pred, target_names=y.columns))
print (accuracy_score(y_test, y_pred))

                        precision    recall  f1-score   support

                 offer       0.00      0.00      0.00        25
           aid_related       0.46      0.12      0.19      2693
          medical_help       0.00      0.00      0.00       516
      medical_products       0.00      0.00      0.00       336
     search_and_rescue       0.00      0.00      0.00       178
              security       0.00      0.00      0.00       120
              military       0.00      0.00      0.00       202
                 water       0.00      0.00      0.00       411
                  food       0.50      0.00      0.01       740
               shelter       0.00      0.00      0.00       558
              clothing       0.00      0.00      0.00       112
                 money       0.00      0.00      0.00       160
        missing_people       0.00      0.00      0.00        64
              refugees       0.00      0.00      0.00       202
                 death       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
sorted(model_pipeline(KNeighborsClassifier()).get_params().keys())

['clf',
 'clf__estimator',
 'clf__estimator__algorithm',
 'clf__estimator__leaf_size',
 'clf__estimator__metric',
 'clf__estimator__metric_params',
 'clf__estimator__n_jobs',
 'clf__estimator__n_neighbors',
 'clf__estimator__p',
 'clf__estimator__weights',
 'clf__n_jobs',
 'memory',
 'steps',
 'tfidf',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'vect',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary']

In [22]:

parameters = {'clf__estimator__n_neighbors': [1,10]
             }
             
cv = GridSearchCV(model_pipeline(KNeighborsClassifier(n_jobs=-1)), param_grid=parameters, verbose=10, cv=2)
cv.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] clf__estimator__n_neighbors=1 ...................................
[CV]  clf__estimator__n_neighbors=1, score=0.23901098901098902, total= 4.4min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.2min remaining:    0.0s


[CV] clf__estimator__n_neighbors=1 ...................................
[CV]  clf__estimator__n_neighbors=1, score=0.20433499542077949, total= 3.8min
[CV] clf__estimator__n_neighbors=10 ..................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 15.7min remaining:    0.0s


[CV]  clf__estimator__n_neighbors=10, score=0.3664021164021164, total= 4.3min
[CV] clf__estimator__n_neighbors=10 ..................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 24.3min remaining:    0.0s


[CV]  clf__estimator__n_neighbors=10, score=0.37193446626640886, total= 4.3min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 32.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 32.8min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...c_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=-1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_neighbors': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:

y_pred = cv.predict(X_test)

print (classification_report(y_test, y_pred, target_names=y.columns))
print (accuracy_score(y_test, y_pred))

                        precision    recall  f1-score   support

                 offer       0.00      0.00      0.00        33
           aid_related       0.47      0.16      0.24      2728
          medical_help       0.00      0.00      0.00       501
      medical_products       0.00      0.00      0.00       344
     search_and_rescue       0.00      0.00      0.00       187
              security       0.00      0.00      0.00       123
              military       0.00      0.00      0.00       219
                 water       0.00      0.00      0.00       404
                  food       0.20      0.00      0.01       753
               shelter       0.00      0.00      0.00       563
              clothing       0.00      0.00      0.00       106
                 money       0.00      0.00      0.00       158
        missing_people       0.00      0.00      0.00        81
              refugees       0.00      0.00      0.00       213
                 death       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [1]:
'''I tried RandomForestClassifier first, it gave me 35% with a low recall.'''


'I tried RandomForestClassifier first, it gave me 35% with a low recall.'

### 9. Export your model as a pickle file

In [13]:
import pickle
filename = 'classifier.pkl'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.